#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# **Word Count Lab: Building a word count application**
This lab will build on the techniques covered in the Spark tutorial to develop a simple word count application.  The volume of unstructured text in existence is growing dramatically, and Spark is an excellent tool for analyzing this type of data.  In this lab, we will write code that calculates the most common words in the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) retrieved from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page).  This could also be scaled to find the most common words on the Internet.

During this lab we will cover: 
- *Part 1:* Creating a base RDD and pair RDDs
- *Part 2:* Counting with pair RDDs
- *Part 3:* Finding unique words and a mean value
- *Part 4:* Apply word count to a file

Note that, for reference, you can look up the details of the relevant methods in [Spark's Python API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

In [0]:
# Homework by 
# Mireia Kesti  NIA: 100406960
# and
# Aleksandra Jamróz NIA: 100491363

## Part 0: Analyze RAW Data
In this part we will create an RDD from the TextFile and check the first elements of it.

In this way, we will analise and check the type of information we will receive.

In [0]:
# get the text file we want to work with and create and RDD of the content 

fileName = "dbfs:/FileStore/shared_uploads/100406960@alumnos.uc3m.es/shakespeare-1.txt"

textRDD = sc.textFile(fileName, 8)


In [0]:
# take the first 10 lines from the textRDD and print them 

textRDD.take(10)
print(textRDD)

dbfs:/FileStore/shared_uploads/100406960@alumnos.uc3m.es/shakespeare-1.txt MapPartitionsRDD[26] at textFile at NativeMethodAccessorImpl.java:0


### ** Part 1: Creating a base RDD and pair RDDs **
In this part of the lab, we will explore creating a base RDD with `parallelize` and using pair RDDs to count words.

#### ** (1a) Create a base RDD **
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

In [0]:
# create a list of words (array)
# create RDD, take 4 words from wordsList array 
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)

# Print out the type of wordsRDD
print (type(wordsRDD))

<class 'pyspark.rdd.RDD'>


#### ** (1b) Pluralize and test **
Let's use a `map()` transformation to add the letter 's' to each string in the base RDD we just created. We'll define a Python function that returns the word with an 's' at the end of the word.  Please replace `<FILL IN>` with your solution.  If you have trouble, the next cell has the solution.  After you have defined `makePlural` you can run the third cell which contains a test.  If you implementation is correct it will print `1 test passed`.

This is the general form that exercises will take, except that no example solution will be provided.  Exercises will include an explanation of what is expected, followed by code cells where one cell will have one or more `<FILL IN>` sections.  The cell that needs to be modified will have `# TODO: Replace <FILL IN> with appropriate code` on its first line.  Once the `<FILL IN>` sections are updated and the code is run, the test cell can then be run to verify the correctness of your solution.  The last code cell before the next markdown section will contain the tests.

In [0]:
#Make a word plural & print it (no proper grammar rules are followed. Just an s is added at the end of the word string)
def makePlural(word):
    
    return word + "s"

print (makePlural('cat'))

cats


In [0]:
test_helper2 = "dbfs:/FileStore/shared_uploads/100406960@alumnos.uc3m.es/test_helper.whl"

In [0]:
# Load in the testing code and check to see if the answer is correct

# first, go to compute --> cluster --> libraries --> install library
# (in my case it kept failing to install it bc I changed the filename. It is important not to change it!!!!)

# If incorrect it will report back '1 test failed' for each failed test. Make sure to rerun any cell you change before trying the test again
from test_helper import Test 

# TEST Pluralize and test (1b)
Test.assertEquals(makePlural('rat'), 'rats', 'incorrect result: makePlural does not add an s')

1 test passed.


#### ** (1c) Apply `makePlural` to the base RDD **
Now wewill pass each item in the base RDD into a [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) transformation that applies the `makePlural()` function to each element. Then we'll call the [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) action to see the transformed RDD.

In [0]:
# Filled in plural function. Applying the makePlural() function to each element
pluralRDD = wordsRDD.map(makePlural)

print (pluralRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


In [0]:
# TEST Apply makePlural to the base RDD(1c)
Test.assertEquals(pluralRDD.collect(), ['cats', 'elephants', 'rats', 'rats', 'cats'],
                  'incorrect values for pluralRDD')

1 test passed.


#### ** (1d) Pass a `lambda` function to `map` **
Let's create the same RDD using a `lambda` function.

In [0]:
# Filled in lambda function to create a plural words RDD
pluralLambdaRDD = wordsRDD.map(lambda x: x + 's')
print (pluralLambdaRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


In [0]:
# TEST Pass a lambda function to map (1d)
Test.assertEquals(pluralLambdaRDD.collect(), ['cats', 'elephants', 'rats', 'rats', 'cats'],
                  'incorrect values for pluralLambdaRDD (1d)')

1 test passed.


#### ** (1e) Length of each word **
Now we'll use `map()` and a `lambda` function to return the number of characters in each word.  We'll `collect` this result directly into a variable.

In [0]:
# Filled in function to obtain the length of each word (using map and lambda)
pluralLengths = (pluralRDD
                 .map(lambda x:len(x))
                 .collect())
print (pluralLengths)

[4, 9, 4, 4, 4]


In [0]:
# TEST Length of each word (1e)
Test.assertEquals(pluralLengths, [4, 9, 4, 4, 4],
                  'incorrect values for pluralLengths')

1 test passed.


#### ** (1f) Pair RDDs **
The next step in writing our word counting program is to create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple `(k, v)` where `k` is the key and `v` is the value. In this example, we will create a pair consisting of `('<word>', 1)` for each word element in the RDD.

We can create the pair RDD using the `map()` transformation with a `lambda()` function to create a new RDD.

In [0]:
# Filled in pair RDD (word pair) function
wordPairs = wordsRDD.map(lambda x: (x, 1))
print (wordPairs.collect())

#output: [('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


In [0]:
# TEST Pair RDDs (1f)
Test.assertEquals(wordPairs.collect(),
                  [('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)],
                  'incorrect value for wordPairs')

1 test passed.


### ** Part 2: Counting with pair RDDs **

Now, let's count the number of times a particular word appears in the RDD. There are multiple ways to perform the counting, but some are much less efficient than others.

A naive approach would be to `collect()` all of the elements and count them in the driver program. While this approach could work for small datasets, we want an approach that will work for any size dataset including terabyte- or petabyte-sized datasets. In addition, performing all of the work in the driver program is slower than performing it in parallel in the workers. For these reasons, we will use data parallel operations.

#### ** (2a) `groupByKey()` approach **
An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions. There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.
 
We'll use `groupByKey()` to generate a pair RDD of type `('word', iterator)`.

In [0]:
# Filled in groupByKey() approach to group the words
# Note that groupByKey requires no parameters
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print ('{0}: {1}'.format(key, list(value)))
    
# output: cat: [1, 1]
# elephant: [1]
# rat: [1, 1]

cat: [1, 1]
elephant: [1]
rat: [1, 1]


In [0]:
# TEST groupByKey() approach (2a)
Test.assertEquals(sorted(wordsGrouped.mapValues(lambda x: list(x)).collect()),
                  [('cat', [1, 1]), ('elephant', [1]), ('rat', [1, 1])],
                  'incorrect value for wordsGrouped')

1 test passed.


#### ** (2b) Use `groupByKey()` to obtain the counts **
Using the `groupByKey()` transformation creates an RDD containing 3 elements, each of which is a pair of a word and a Python iterator.
Now sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

In [0]:
# Filled in pair of word, count RDD 
# keep getting error "invalid syntax" for: (lambda x, y: (x, sum(y))) --> bc it does not want to split the tuple
# fixed the error by changing it to the following (it is performing the same split, but it does not realize it)
wordCountsGrouped = wordsGrouped.map(lambda x: (x[0], sum(x[1])))
print (wordCountsGrouped.collect())
print(wordsGrouped.take(2))

[('cat', 2), ('elephant', 1), ('rat', 2)]
[('cat', <pyspark.resultiterable.ResultIterable object at 0x7f8a6d7b95e0>), ('elephant', <pyspark.resultiterable.ResultIterable object at 0x7f8a6c559be0>)]


In [0]:
# TEST Use groupByKey() to obtain the counts (2b)
Test.assertEquals(sorted(wordCountsGrouped.collect()),
                  [('cat', 2), ('elephant', 1), ('rat', 2)],
                  'incorrect value for wordCountsGrouped')

1 test passed.


#### ** (2c) Counting using `reduceByKey` **

A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

In [0]:
# filled in function to count the number of words
# Note that reduceByKey takes in a function that accepts two values and returns a single value

# if the lambda function does not work, try replacing it with (lambda (x,y): x+y)
# the documentation states that the () should be there, but it returns an error if I write them. It works fine without them.

wordCounts = wordPairs.reduceByKey(lambda x,y: x+y)
print (wordCounts.collect())

#output: [('cat', 2), ('elephant', 1), ('rat', 2)]

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [0]:
# TEST Counting using reduceByKey (2c)
Test.assertEquals(sorted(wordCounts.collect()), [('cat', 2), ('elephant', 1), ('rat', 2)],
                  'incorrect value for wordCounts')

1 test passed.


#### ** (2d) All together **

The expert version of the code performs the `map()` to pair RDD, `reduceByKey()` transformation, and `collect` in one statement.

In [0]:
# filled in function to use map() to pair RDD, reduceByKey() transformation, and collect in one statement.
wordCountsCollected = (wordsRDD.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
                       .collect())
print (wordCountsCollected)

#output: [('cat', 2), ('elephant', 1), ('rat', 2)]

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [0]:
# TEST All together (2d)
Test.assertEquals(sorted(wordCountsCollected), [('cat', 2), ('elephant', 1), ('rat', 2)],
                  'incorrect value for wordCountsCollected')

1 test passed.


### ** Part 3: Finding unique words and a mean value **

#### ** (3a) Unique words **

Now, we'll calculate the number of unique words in `wordsRDD`.  Other RDDs that we have already created can be used to make this task easier.

In [0]:
# Filled in function to count the number of unique words
uniqueWords = wordsRDD.distinct().count()
print (uniqueWords)

# output: 3

3


In [0]:
# TEST Unique words (3a)
Test.assertEquals(uniqueWords, 3, 'incorrect count of uniqueWords')

1 test passed.


#### ** (3b) Mean using `reduce` **

Now we will compute the mean number of words per unique word in `wordCounts`.

We will use a `reduce()` action to sum the counts in `wordCounts` and then divide by the number of unique words.  First `map()` the pair RDD `wordCounts`, which consists of (key, value) pairs, to an RDD of values.

In [0]:
# TODO: Replace <FILL IN> with appropriate code
from operator import add
#error: says wordCounts is not defined. This code won't work unless the issues in 2b, 2c are solved (bc that is when we are declaring this variable)

totalCount = (wordCounts
              .map(lambda x: x[1])
              .reduce(add))
average = totalCount / float(uniqueWords)

print (totalCount) #output: 5
print (round(average, 2)) #output: 1.67

5
1.67


In [0]:
# TEST Mean using reduce (3b)
Test.assertEquals(round(average, 2), 1.67, 'incorrect value of average')

1 test passed.


### ** Part 4: Apply word count to a file **

In this section we will finish developing our word count application.  We'll have to build the `wordCount` function, deal with real world problems like capitalization and punctuation, load in our data source, and compute the word count on the new data.

#### ** (4a) `wordCount` function **

First, we'll define a function for word counting.  You should reuse the techniques that have been covered in earlier parts of this lab.  This function should take in an RDD that is a list of words like `wordsRDD` and return a pair RDD that has all of the words and their associated counts.

In [0]:
#Filled in function to count the times the word appears 
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return wordListRDD.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
print (wordCount(wordsRDD).collect())

#output: [('cat', 2), ('elephant', 1), ('rat', 2)]

[('cat', 2), ('elephant', 1), ('rat', 2)]


In [0]:
# TEST wordCount function (4a)
Test.assertEquals(sorted(wordCount(wordsRDD).collect()),
                  [('cat', 2), ('elephant', 1), ('rat', 2)],
                  'incorrect definition for wordCount function')

1 test passed.


#### ** (4b) Capitalization and punctuation **

Real-world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.
 
First, we'll define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.

In [0]:
# filled in function to remove punctuation
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    
    t1 = text.lower()
    t2 = re.sub(r'[^0-9a-z\s]',"",t1)
    t3 = t2.strip()
    
    return t3

print (removePunctuation('Hi, you!')) #output: hi you
print (removePunctuation(' No under_score!')) #output: no underscore

hi you
no underscore


In [0]:
# TEST Capitalization and punctuation (4b)
Test.assertEquals(removePunctuation(" The Elephant's 4 cats. "),
                  'the elephants 4 cats',
                  'incorrect definition for removePunctuation function')

1 test passed.


#### ** (4c) Load a text file **

For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lowercase.  Since the file is large we use `take(15)`, so that we only print 15 lines.

In [0]:
# Just run this code
import os.path
fileName = "dbfs:/FileStore/shared_uploads/100406960@alumnos.uc3m.es/shakespeare-1.txt"

shakespeareRDD = (sc
                  .textFile(fileName, 8)
                  .map(removePunctuation))
print ('\n'.join(shakespeareRDD
                .zipWithIndex()  # to (line, lineNum)
                .map(lambda data: '{0}: {1}'.format(data[0], data[1]))  # to 'lineNum: line'
                .take(15)))

"""
output:
1609: 0
: 1
the sonnets: 2
: 3
by william shakespeare: 4
: 5
: 6
: 7
1: 8
from fairest creatures we desire increase: 9
that thereby beautys rose might never die: 10
but as the riper should by time decease: 11
his tender heir might bear his memory: 12
but thou contracted to thine own bright eyes: 13
feedst thy lights flame with selfsubstantial fuel: 14
"""

1609: 0
: 1
the sonnets: 2
: 3
by william shakespeare: 4
: 5
: 6
: 7
1: 8
from fairest creatures we desire increase: 9
that thereby beautys rose might never die: 10
but as the riper should by time decease: 11
his tender heir might bear his memory: 12
but thou contracted to thine own bright eyes: 13
feedst thy lights flame with selfsubstantial fuel: 14
Out[47]: '\noutput:\n1609: 0\n: 1\nthe sonnets: 2\n: 3\nby william shakespeare: 4\n: 5\n: 6\n: 7\n1: 8\nfrom fairest creatures we desire increase: 9\nthat thereby beautys rose might never die: 10\nbut as the riper should by time decease: 11\nhis tender heir might bear his memory: 12\nbut thou contracted to thine own bright eyes: 13\nfeedst thy lights flame with selfsubstantial fuel: 14\n'

#### ** (4d) Words from lines **

Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + The first issue is that  that we need to split each line by its spaces.
  + The second issue is we need to filter out empty lines.
 
We'll aqpply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, we should apply Python's string [split()](https://docs.python.org/2/library/string.html#string.split) function. (We could think that a `map()` transformation is the way to do this, but we should think about what the result of the `split()` function will be.)

In [0]:
# filled in function to split each element of the RDD by its spaces 

shakespeareWordsRDD = shakespeareRDD.flatMap(lambda x: x.split(" "))
shakespeareWordCount = shakespeareWordsRDD.count()

print (shakespeareWordsRDD.top(5)) #output: ['zwaggerd', 'zounds', 'zounds', 'zounds', 'zounds']
print (shakespeareWordCount) #output: 927631

['zwaggerd', 'zounds', 'zounds', 'zounds', 'zounds']
927631


In [0]:
# TEST Words from lines (4d)
# This test allows for leading spaces to be removed either before or after
# punctuation is removed.
Test.assertTrue(shakespeareWordCount == 927631 or shakespeareWordCount == 928908 or shakespeareWordCount == 882996,
                'incorrect value for shakespeareWordCount')
Test.assertEquals(shakespeareWordsRDD.top(5),
                  [u'zwaggerd', u'zounds', u'zounds', u'zounds', u'zounds'],
                  'incorrect value for shakespeareWordsRDD')

1 test passed.
1 test passed.


#### ** (4e) Remove empty elements **

The next step is to filter out the empty elements.  We will now remove all entries where the word is `''`.

In [0]:
# filled in function to filter out empty elements
shakeWordsRDD = shakespeareWordsRDD.filter(lambda x: x!='')
shakeWordCount = shakeWordsRDD.count()

print (shakeWordCount) #output: 882996

882996


In [0]:
# TEST Remove empty elements (4e)
Test.assertEquals(shakeWordCount, 882996, 'incorrect value for shakeWordCount')

1 test passed.


#### ** (4f) Count the words **

We now have an RDD that is only words.  Next, we'll apply the `wordCount()` function to produce a list of word counts. We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pairs, we need a custom sort function that sorts using the value part of the pair.

We notice that many of the words are common English words (stopwords). In a later lab, we will see how to eliminate them from the results.

We'll use the `wordCount()` function and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [0]:
# filled in function to count the top 15 words & their count
top15WordsAndCounts = wordCount(shakeWordsRDD).sortBy(lambda x: x[1], ascending=False).take(15) 

print ('\n'.join(map(lambda data: '{0}: {1}'.format(data[0], data[1]), top15WordsAndCounts)))

"""
output: 
the: 27361
and: 26028
i: 20681
to: 19150
of: 17463
a: 14593
you: 13615
my: 12481
in: 10956
that: 10890
is: 9134
not: 8497
with: 7771
me: 7769
it: 7678
"""

the: 27361
and: 26028
i: 20681
to: 19150
of: 17463
a: 14593
you: 13615
my: 12481
in: 10956
that: 10890
is: 9134
not: 8497
with: 7771
me: 7769
it: 7678
Out[52]: '\noutput: \nthe: 27361\nand: 26028\ni: 20681\nto: 19150\nof: 17463\na: 14593\nyou: 13615\nmy: 12481\nin: 10956\nthat: 10890\nis: 9134\nnot: 8497\nwith: 7771\nme: 7769\nit: 7678\n'

In [0]:
# TEST Count the words (4f)
Test.assertEquals(top15WordsAndCounts,
                  [(u'the', 27361), (u'and', 26028), (u'i', 20681), (u'to', 19150), (u'of', 17463),
                   (u'a', 14593), (u'you', 13615), (u'my', 12481), (u'in', 10956), (u'that', 10890),
                   (u'is', 9134), (u'not', 8497), (u'with', 7771), (u'me', 7769), (u'it', 7678)],
                  'incorrect value for top15WordsAndCounts')

1 test passed.


### Conclusion
In this lab, and to be able to understand what we are doing, we have learned what RDDs are, and what they are used for.
Resilient Distributed Dataset (RDD) is a core data structure of PySpark. It is a low-level object & highly efficient in performing distributed tasks. At the core, an RDD is an immutable distributed collection of elements of data, partitioned across nodes in the cluster that can be operated in parallel with a low-level API that offers transformations & actions.
RDDs are useful when we want to do low-level transformations on our dataset and when our data is unstructured, such as streams of text, like in this lab.

We have also consulted the Spark Apache documentation and realised that the syntax it recommends is not always the one that works. This is (probably) due to the different Python versions between the current session on Databricks and the last update to the official documentation. However, by trying alternate versions of inputting the same code, we were able to solve the issue.

Furthermore, it is important to remember to attach the cluster, and to install the test library on the same cluster. To be able to correctly install the library, its filename cannot be modified.